### Step 1: Install necesscary packages

In [33]:
!pip install matplotlib
!pip install torch numpy transformers datasets tiktoken wandb tqdm

In [34]:
!pip3 install tqdm

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
%cd /content/drive/MyDrive/Colab Notebooks/SC3000/dpo

/content/drive/MyDrive/Colab Notebooks/SC3000/dpo


### Step 2: Package imports and configuration

In [37]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import random
import pickle
from model import GPT, GPTConfig
from tqdm import tqdm
import time
import json
import matplotlib.pyplot as plt

torch.manual_seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)


# Configuration
beta = 0.5
if torch.cuda.is_available():
    device = "cuda"
    print("cuda")
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

base_lr = 0.0001
epochs = 20
batch_size = 64
max_length =64
num_samples = 1
max_new_tokens = 200
temperature = 0.8
top_k = 200
# tokenizer
with open("../sft/meta.pkl", "rb") as f:
    meta = pickle.load(f)
stoi, itos = meta["stoi"], meta["itos"]
#def encode(s): return [stoi[c] for c in s]
def encode(s):
    return [stoi[c] if c in stoi else 0 for c in s]
def decode(l): return ''.join([itos[i] for i in l])

cuda


### Step 3: Define helper functions

In [38]:
def compute_logprob(input_ids):
    inputs = input_ids[:, :-1]
    targets = input_ids[:, 1:]
    logits, _ = gpt(inputs, full_seq=True)
    B, T, V = logits.size()
    logits_flat = logits.reshape(-1, V)
    targets_flat = targets.reshape(-1)
    loss = F.cross_entropy(logits_flat, targets_flat, ignore_index=0, reduction='none')
    loss = loss.reshape(B, T)
    attention_mask = (targets != 0).float()
    loss = (loss * attention_mask).sum(dim=1) / attention_mask.sum(dim=1)
    return -loss

def pad_or_truncate(seq, max_length):
    return seq[-max_length:] if len(seq) > max_length else seq + [0] * (max_length - len(seq))

def get_batches(lines, batch_size):
    random.shuffle(lines)
    #for l in lines:
    #    print(l[1])
    for i in range(0, len(lines), batch_size):
        batch = lines[i:i+batch_size]
        if len(batch) < batch_size:
            continue
        neg_inputs = [pad_or_truncate(encode(p['negative'] + '\n\n\n\n'), max_length) for p in batch]
        pos_inputs = [pad_or_truncate(encode(p['positive'] + '\n\n\n\n'), max_length) for p in batch]
        neg_tensor = torch.tensor(neg_inputs, dtype=torch.long, device=device)
        pos_tensor = torch.tensor(pos_inputs, dtype=torch.long, device=device)
        yield neg_tensor, pos_tensor

### Step 4: Load the pretrained NanoGPT model

In [39]:
ckpt = torch.load("../sft/gpt.pt", map_location=device)
gptconf = GPTConfig(**ckpt['model_args'])
gpt = GPT(gptconf)
state_dict = ckpt['model']
unwanted_prefix = '_orig_mod.'
for k in list(state_dict.keys()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
gpt.load_state_dict(state_dict)
gpt.to(device).train()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(74, 348)
    (wpe): Embedding(256, 348)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=348, out_features=1044, bias=False)
          (c_proj): Linear(in_features=348, out_features=348, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=348, out_features=1392, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1392, out_features=348, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=348, out_features=74, bias=False)
)

### Step 5: Load Data (**students are required to complete this part!**)

In [ ]:
# Load data from ./data/pos_neg_pairs.json
with open('/content/drive/MyDrive/Colab Notebooks/SC3000/data/pos_neg_pairs_1mil.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    lines = data

# Check the structure
print(type(data))
print(len(data))
print(data[1])

<class 'list'>
750000
{'negative': '197-2=? Sorry, I do not know!', 'positive': '197-2=? The answer is 195 because 197-2 equals 195.'}


### Step 6: Build the optimizer and scheduler (**students are required to complete this part!**)

In [41]:
def build_optimizer_and_scheduler(
    model,
    lr=base_lr,
    weight_decay=0.05,
):
    # ---- no weight decay for bias & LayerNorm/BatchNorm ----
    decay, no_decay = [], []
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        if p.ndim >= 2:
            decay.append(p)
        else:
            no_decay.append(p)

    optim_groups = [
        {"params": decay, "weight_decay": weight_decay},
        {"params": no_decay, "weight_decay": 0.0},
    ]

    # AdamW optimizer
    optimizer = AdamW(optim_groups, lr=lr)

    # Step Decay scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=len(lines) // batch_size * epochs, eta_min=1e-6
)

    return optimizer, scheduler


In [42]:
optimizer, scheduler = build_optimizer_and_scheduler(
    gpt,
    lr=base_lr,
    weight_decay=0.05,
)

### Step 7: Begin training (**students are required to complete this part!**)

In [43]:
gpt.to(device).train()

total_steps = len(lines) // batch_size
for epoch in range(epochs):
    pbar = tqdm(get_batches(lines, batch_size))
    for step, (neg_tensor,pos_tensor) in enumerate(pbar):
        ###########################################################
        # Please complete the training code here!
        # Examples:
        # ...
        # neg_logprob
        # pos_logprob
        # loss = -F.logsigmoid((pos_logprob - neg_logprob) / beta).mean() - pos_logprob.mean() * 0.1
        # ...
        pos_logprob = compute_logprob(pos_tensor)
        neg_logprob = compute_logprob(neg_tensor)
        loss = -F.logsigmoid((pos_logprob - neg_logprob) / beta).mean()- pos_logprob.mean() * 0.1
        optimizer.zero_grad(set_to_none=True)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(gpt.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        if (step + 1) % 10 == 0:
            pbar.set_postfix(loss=float(loss.item()), lr=scheduler.get_last_lr()[0])

        ###########################################################
    ckpt_path = f"./dpo.pt"
    torch.save({
        "model_state_dict": gpt.state_dict(),
        "model_args": ckpt['model_args'],
    }, ckpt_path)
    print(f"Saved checkpoint to {ckpt_path}")

11718it [07:58, 24.46it/s, loss=0.0313, lr=9.94e-5]


Saved checkpoint to ./dpo.pt


11718it [07:58, 24.51it/s, loss=0.0271, lr=9.76e-5]


Saved checkpoint to ./dpo.pt


11718it [07:57, 24.54it/s, loss=0.0255, lr=9.46e-5]


...

Saved checkpoint to ./dpo.pt


11718it [07:57, 24.54it/s, loss=0.0245, lr=5.05e-5]


Saved checkpoint to ./dpo.pt


8815it [05:59, 24.55it/s, loss=0.0237, lr=4.47e-5]


KeyboardInterrupt: 

# Step 8: Begin testing (**students are required to complete this part!**)

In [ ]:
# Load the fine-tuned model
ckpt_path = "/content/drive/MyDrive/Colab Notebooks/SC3000/dpo/dpo.pt"
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
gpt = GPT(gptconf).to(device)
try:
    state_dict = checkpoint['model']
except:
    state_dict = checkpoint['model_state_dict']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
gpt.load_state_dict(state_dict)
# Test
gpt.eval()
test_set = ["17+19=?", "3*17=?", "72/4=?", "72-x=34,x=?", "x*11=44,x=?", "3*17=?", "72/4=?", "72-x=34,x=?"]

with torch.no_grad():
    for prompt in test_set:
        prompt_ids = encode(prompt)
        ###########################################################
        # Please complete the test code here!
        # ...
        # gpt.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
        # ...
        x = torch.tensor([prompt_ids], dtype=torch.long, device=device)  # shape [1, T]

        y = gpt.generate(
            x,
            max_new_tokens=max_new_tokens,      # you defined these above
            temperature=temperature,
            top_k=top_k
        )
        out_full = decode(y[0].cpu().flatten().tolist())
        generated = out_full[len(prompt):].strip()

        print(f"Q: {prompt}")
        print(f"A: {generated}\n")
        ###########################################################

Q: 17+19=?
A: The answer is 36 because 17+19 equals 36.

Q: 3*17=?
A: The answer is 51 because 3*17 equals 51.

Q: 72/4=?
A: The answer is 18 because 72/4 equals 18.

Q: 72-x=34,x=?
A: The answer is 38 because 72-34 equals 38.

Q: x*11=44,x=?
A: The answer is 44 because 44/11 equals 44.

Q: 3*17=?
A: The answer is 51 because 3*17 equals 51.

Q: 72/4=?
A: The answer is 18 because 72/4 equals 18.

Q: 72-x=34,x=?
A: The answer is 38 because 72-34 equals 38.

